In [10]:
#vkrishn7 200417228
from qiskit import Aer, QuantumCircuit, execute
from qiskit import IBMQ
#IBMQ.save_account(YOUR-TOKEN-HERE)
IBMQ.save_account('6ad660c99fd7054787da3cd13d29873bad4c0c901f2e0653852c11584a8a6398fb47de4b810bceeb78597a9ce3c4b18d198b38f56061a7ef44c67b2742484432', overwrite=True)
IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-11-05 17:48:54,891: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [11]:
def dotprod(a, b):
    temp_sum = 0
    for i in range(len(a)):
        temp_sum += int(a[i])*int(b[i])
    return temp_sum%2

In [12]:
def bv(s,n,qc):
   qc.x(n) 
   qc.barrier()
   qc.h(range(n+1))
   qc.barrier()
   for i,yes in enumerate(reversed(s)):
      if yes == '1': 
        qc.cx(i, n)
        
   qc.barrier()
   qc.h(range(n+1))
   qc.barrier()

In [13]:
def oracle(circ, b):
    # TODO generalize this function
    for i, char in enumerate(b):
        circ.cx(i, i+len(b))
    for i,yes in enumerate(reversed(b)):
      if yes == '1': 
        circ.cx(len(b)-1, i+len(b))
    #circ.cx(1, 4)
    #circ.cx(1, 5)

In [14]:
# Main function
b = '1111'
n = len(b)
run_bv = 1
run_simon = 1
circ_bv = QuantumCircuit(n*2, n*2)

In [15]:
bv(b,len(b),circ_bv)

In [16]:
circ_mid_mes_rst = QuantumCircuit(n*2,n*2)
circ_mid_mes_rst.reset(range(n*2))
circ_mid_mes_rst.barrier()

circ_simon = QuantumCircuit(n*2,n*2)
circ_simon.h(range(n))
circ_simon.barrier()
oracle(circ_simon, b)
circ_simon.barrier()
circ_simon.h(range(n))
circ_simon.measure(range(n), range(n,n*2))

In [17]:
if(run_bv == 1 and run_simon == 0):
   circ_bv.measure(range(n),range(n))
   circ_net = circ_bv
if(run_bv == 0 and run_simon == 1):
    circ_net = circ_mid_mes_rst.compose(circ_simon)
if(run_bv == 1 and run_simon == 1):
   circ_bv.measure(range(n),range(n))
   circ_net = circ_bv.compose(circ_mid_mes_rst)
   circ_net = circ_net.compose(circ_simon)
    
print(circ_net)

#simulator = Aer.get_backend('aer_simulator')
#results = execute(circ_net, backend=simulator, shots=512).result()
#counts = results.get_counts()
#from qiskit.visualization import plot_histogram
#plot_histogram(result.get_counts(circ_net))
 

           ░ ┌───┐ ░                      ░ ┌───┐ ░ ┌─┐                     ░ »
q_0: ──────░─┤ H ├─░───■──────────────────░─┤ H ├─░─┤M├─|0>─────────────────░─»
           ░ ├───┤ ░   │                  ░ ├───┤ ░ └╥┘ ┌─┐                 ░ »
q_1: ──────░─┤ H ├─░───┼────■─────────────░─┤ H ├─░──╫──┤M├──|0>────────────░─»
           ░ ├───┤ ░   │    │             ░ ├───┤ ░  ║  └╥┘  ┌─┐            ░ »
q_2: ──────░─┤ H ├─░───┼────┼────■────────░─┤ H ├─░──╫───╫───┤M├──|0>───────░─»
           ░ ├───┤ ░   │    │    │        ░ ├───┤ ░  ║   ║   └╥┘  ┌─┐       ░ »
q_3: ──────░─┤ H ├─░───┼────┼────┼────■───░─┤ H ├─░──╫───╫────╫───┤M├──|0>──░─»
     ┌───┐ ░ ├───┤ ░ ┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐ ░ ├───┤ ░  ║   ║    ║   └╥┘       ░ »
q_4: ┤ X ├─░─┤ H ├─░─┤ X ├┤ X ├┤ X ├┤ X ├─░─┤ H ├─░──╫───╫────╫────╫───|0>──░─»
     └───┘ ░ └───┘ ░ └───┘└───┘└───┘└───┘ ░ └───┘ ░  ║   ║    ║    ║        ░ »
q_5: ──────░───────░──────────────────────░───────░──╫───╫────╫────╫───|0>──░─»
           ░       ░                    

In [18]:

#IBMQ.get_provider(hub='ibm-q')
#backend = provider.get_backend('ibmq_ithaca')

provider = IBMQ.get_provider(hub='ibm-q-ncsu', group='nc-state', project='grad-qc-class')
#backend = provider.get_backend('ibmq_mumbai')

backend = Aer.get_backend('aer_simulator')
shots = 1024
job = execute(circ_net, backend=backend, shots=shots)
results = job.result()
counts = results.get_counts()

if(run_bv == 1):
   print(counts)

if(run_simon == 1):
   for code in counts:
      print("(" + str(b) + " dot " + str(code) + ")%2 = " + str(dotprod(b, code)))

{'01011111': 123, '10011111': 137, '00111111': 135, '10101111': 124, '11111111': 127, '00001111': 124, '01101111': 130, '11001111': 124}
(1111 dot 01011111)%2 = 0
(1111 dot 10011111)%2 = 0
(1111 dot 00111111)%2 = 0
(1111 dot 10101111)%2 = 0
(1111 dot 11111111)%2 = 0
(1111 dot 00001111)%2 = 0
(1111 dot 01101111)%2 = 0
(1111 dot 11001111)%2 = 0


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
